# Chemical equilibrium with constraints

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 6th, 2022</p>

As we mentioned in the previous section, Reaktoro can be used for a wide range of chemical equilibrium problems. In this section, we show:

* the fundamentals of specifying different equilibrium constraints when calculating chemical equilibrium; and
* how the equilibrium solver can be configured to allow the system to be open to one or more substances.

We learned that a chemical equilibrium solver can be created with the class {{EquilibriumSolver}}, as shown in the example below:

In [2]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

solver = EquilibriumSolver(system)

{{EquilibriumSolver}} objects created this way (i.e., as in `solver = EquilibriumSolver(system)`), are limited to chemical equilibrium calculations where:

* temperature and pressure are given;
* the system is closed (no mass transfer in or out of the system).

Thus, `solver = EquilibriumSolver(system)` creates a **Gibbs energy minimization solver**. 

This is not always the desired behavior, however. If we want to impose the temperature and volume of the system, we will need to construct a specialized {{EquilibriumSolver}} object that can handle these constraint specifications.

```{note}
Chemical equilibrium problems in which temperature and volume are prescribed belong to the class of *Helmholtz energy minimization problems* {cite}`Smith1982`. Reaktoro, however, does not implement a Helmholtz energy minimization solver. Instead, Reaktoro implements a single **parametric Gibbs energy minimization solver that accepts general equilibrium constraints**, which can be configured to efficiently solve all other classes of equilibrium problems (including Helmholtz energy minimization problems!).
```

## Specifying constrained properties

Let's create an equilibrium solver that can solve problems with prescribed **temperature** and **volume**:

In [3]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.volume()

solver = EquilibriumSolver(specs)

That's it! We use {{EquilibriumSpecs}} to provide the constraint specifications for our {{EquilibriumSolver}}.

```{tip}
Access the link {{EquilibriumSpecs}} to find out all currently existing constraints you can impose! It is even **possible to define your own constraints**, instead of predefined ones, which we'll cover in a dedicated tutorial given this is a more advanced use case. 
```

It's now time to demonstrate how we use this specialized equilibrium solver. We want to model the combustion of 1 mol of CH{{_4}} and 0.5 mol of O{{_2}} in a chamber of 10 cm³ at 1000 °C. We create below an initial chemical state with this gas composition:

In [4]:
state = ChemicalState(system)
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |   298.1500 |    K |
| Pressure        |     1.0000 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.0000e+00 |  mol |
| :: C            | 1.0000e+00 |  mol |
| :: O            | 1.0000e+00 |  mol |
| Species Amount: |            |      |
| :: CH4          | 1.0000e+00 |  mol |
| :: O2           | 5.0000e-01 |  mol |
| :: CO2          | 1.0000e-16 |  mol |
| :: CO           | 1.0000e-16 |  mol |
| :: H2O          | 1.0000e-16 |  mol |
| :: H2           | 1.0000e-16 |  mol |
+-----------------+------------+------+


This state is in disequilibrium. We use {{EquilibriumConditions}} below to specify the desired conditions of temperature and volume at the equilibrium state, and then equilibrate the `state` object:

In [5]:
conditions = EquilibriumConditions(specs)
conditions.temperature(1000.0, "celsius")
conditions.volume(10.0, "cm3")

solver.solve(state, conditions)

print("FINAL STATE")
print(state)

FINAL STATE
+-----------------+------------+------+
| Property        |      Value | Unit |
+-----------------+------------+------+
| Temperature     |  1273.1500 |    K |
| Pressure        | 16603.2449 |  bar |
| Charge:         | 0.0000e+00 |  mol |
| Element Amount: |            |      |
| :: H            | 4.0000e+00 |  mol |
| :: C            | 1.0000e+00 |  mol |
| :: O            | 1.0000e+00 |  mol |
| Species Amount: |            |      |
| :: CH4          | 7.1576e-01 |  mol |
| :: O2           | 1.0000e-16 |  mol |
| :: CO2          | 2.2509e-01 |  mol |
| :: CO           | 5.9148e-02 |  mol |
| :: H2O          | 4.9067e-01 |  mol |
| :: H2           | 7.7814e-02 |  mol |
+-----------------+------------+------+


This printed state does not show volume. Let's check the volume in the {{ChemicalProps}} object attached by {{EquilibriumSolver}} to the {{ChemicalState}} object `state` at the end of the computation. Let's also print the computed pressure:

In [6]:
V = state.props().volume()  # volume in m³
P = state.pressure()        # pressure in Pa

V = units.convert(V, "m3", "cm3")  # convert volume from m³ to cm³
P = units.convert(P, "Pa", "GPa")  # convert pressure from Pa to GPa

print("Volume at equilibrium state is", V, "cm³!")
print("Pressure at equilibrium state is", P, "GPa!")

Volume at equilibrium state is 10 cm³!
Pressure at equilibrium state is 1.66032 GPa!


Note that the volume at the computed equilibrium state is exactly what we enforced in the {{EquilibriumConditions}} object: 10 cm³.

```{attention}
Don't forget to set the expected conditions in the {{EquilibriumConditions}} object. For every condition marked to be known in the {{EquilibriumSpecs}} object (e.g., `specs.temperature()` and `specs.volume()`), make sure you give a value for each of them in the associated {{EquilibriumConditions}} object (e.g., `conditions.temperature(1000.0, "celsius")` and `conditions.volume(10.0, "cm3")`)! Not doing this can cause unexpected behavior or a runtime error.
```

```{admonition} Did you know?
The Gibbs energy minimization computation performed with:

~~~py
solver = EquilibriumSolver(system)
solver.solve(state)
~~~

is executed in the background with the following code:

~~~py
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()

conditions = EquilibriumConditions(specs)
conditions.temperature(state.temperature())
conditions.pressure(state.pressure())

solver = EquilibriumSolver(specs)
solver.solve(state, conditions)
~~~

so you can type a lot less if all you need is equilibrium calculations with specified temperatures and pressures! 
```

## Specifying open conditions

Let's say we would like to find out how many mols of CH{{_4}} must react with 0.5 mol of O{{_2}} in the same chamber (with volume 10 cm³ and temperature 1000 °C) to obtain pressure 1 GPa (which, let's assume, is the maximum pressure supported by the chamber material).

In this new equilibrium calculation, the system is open to the mass transfer of CH{{_4}}, and this creates a new *degree of freedom* in the problem: the amount added/removed of this substance. That's why we can simultaneously impose a value for temperature (1000 °C), volume (10 cm³), and pressure (1 GPa), which would be thermodynamically difficult otherwise.

The code below starts by creating a chemical state in which there is only 0.5 mols of O{{_2}}:

In [7]:
state = ChemicalState(system)
state.set("O2", 0.5, "mol")

Next, we create an {{EquilibriumSpecs}} object in which we specify that **temperature**, **volume**, and **pressure** are constrained properties. We also specify that the system is open to CH{{_4}}. This is all followed by the creation of a specialized and optimized {{EquilibriumSolver}} object to handle this kind of equilibrium calculations:

In [8]:

specs = EquilibriumSpecs(system)
specs.temperature()
specs.volume()
specs.pressure()
specs.openTo("CH4")

solver = EquilibriumSolver(specs)

We can now create the {{EquilibriumConditions}} object in which we provide the values for temperature, volume, and pressure, and then perform the calculation:

In [9]:

conditions = EquilibriumConditions(specs)
conditions.temperature(1000.0, "celsius")
conditions.volume(10.0, "cm3")
conditions.pressure(1.0, "GPa")

solver.solve(state, conditions)

print(state.props())

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |    1273.1500 |         K |
| Pressure                               |   10000.0000 |       bar |
| Volume                                 |   1.0000e-05 |        m3 |
| Gibbs Energy                           | -488420.1304 |         J |
| Enthalpy                               | -184792.5765 |         J |
| Entropy                                |     238.4853 |       J/K |
| Internal Energy                        | -194792.5765 |         J |
| Helmholtz Energy                       | -498420.1304 |         J |
| Charge                                 |   0.0000e+00 |       mol |
| Element Amount:                        |              |           |
| :: H                                   |   1.5995e+00 |       mol |
| :: C              

The above table shows that we obtained an equilibrium state with desired values for temperature, pressure, and volume (in SI units). We can also see below the amount of CH{{_4}} that entered the system so that all these prescribed conditions could be attained:

In [12]:
print(f"Amount of CH4 titrated in: {state.equilibrium().titrantAmount('CH4')} mol")

Amount of CH4 titrated in: 0.399884 mol


## Recap

Reaktoro can solve chemical equilibrium problems with a variety of specifications. This guide demonstrated how this can be done for some relatively simple cases using classes:

* {{EquilibriumSolver}}
* {{EquilibriumSpecs}}
* {{EquilibriumConditions}}

We will later learn how these classes can be used to model more complicated equilibrium problems. 